# <mark>Processa pipeline models</mark>

### Import

In [1]:
import os
import csv
import json
import shutil
from io import StringIO
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

import cv2
from PIL import Image
import pytesseract

import fitz  # Módulo PyMuPDF
import re
from fuzzywuzzy import fuzz
from unidecode import unidecode
from unicodedata import normalize
import PyPDF2

import pandas as pd

from PIL import ImageFont
from PIL import Image, ImageDraw

from datetime import datetime, timezone, timedelta

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar



### Importa modulos internos

In [2]:
import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron
import modules.trata_dicionarios as tratdic
import modules.trata_imagem as tratimg
import modules.trata_files as tratfiles
import modules.trata_texto_extraido as trattext
import modules.trata_template as trattemplate


NameError: name 'name_image_2work' is not defined

### Config

In [6]:
### PRESTAR ATENCAO
model = 'pedro_aldeia'


#### IMPORTANTE - Nro Batch
batch_name = "Batch_9"


# 1. path para documentos PDF (omelhor se estiverem dentro de um unico diretorio)
root_pdf_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 2. path para documentos PDF que podem estar aguardando para serem processados
root_pdf_aguardando_path = "pipeline_extracao_documentos/3_tratamento_excecoes/pdf_aguardando_processar"

documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 3. path para documentos PDF externos para serem processados
root_external_pdf_path = "content_from_pdftool/data/data_pdf/NF_para_processamento/NFRJ_PDF_para _ocr"
# 4. path para documentos PDF PESQUISAVEIS externos para serem processados
root_external_pdf_pesquisavel_path = "content_from_pdftool/data/data_pdf/NF_processadas/NFRJ/fwdnotasfiscaisemitidaslmpadalegal"

# 5. path para imagem padrao
image_resized_path = 'pipeline_extracao_documentos/6_geral_administacao/images/processadas'

# 6. path para log
log_path = 'pipeline_extracao_documentos/6_geral_administacao/logs'

# 7. path para arquivos json
json_path = "pipeline_extracao_documentos/5_documentos_processados/jsons"

# 8. path para NFs processadas
nf_processada_path = "pipeline_extracao_documentos/5_documentos_processados"

#### paths de objetos para criacao/gestao (dicionarios/datasets)
# 9. path para models
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v9.xlsx"

# 10. path para dicionario de models
model_dict_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/models.csv"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"



# VERIFICAR
tgt_imagens = "pipeline_extracao_documentos/6_geral_administacao/images"

# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'


## Processamento

In [9]:
#### IMPORTANTE - NRO BATCH PARA TESTE    0 = PDF_PESQUISAVEL | 1 = RASTER_PDF

i_test = 1

tipo_pdf = []
tipo_pdf.append('PDF_PESQUISAVEL')
tipo_pdf.append('RASTER_PDF')
tipo_pdf[i_test]


# Tratamento do Path de ORIGEM DO DOCUMENTOS PARA TESTE QUE SERAO MOVIDOS
list_path_test = []
list_path_test.append("pipeline_extracao_documentos/4_area_testes/pdf_pesquisavel_4_test")
list_path_test.append("pipeline_extracao_documentos/4_area_testes/raster_pdf_4_test")
list_path_test[i_test]

# Frame para teste
i_frame = 0

frames_pesquisa = []
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == model]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_name = row_frame['label']
    frames_pesquisa.append(frame_name)

# Nome Batch
batch_name = "Batch_" + str(tipo_pdf[i_test]) + "_" + str(i_frame)

# Nome do arquivo json
nome_formado_json = batch_name +".json"

# Listagem dos frames de pesquisa
i = 0
for frame in frames_pesquisa:
    print(f'seq ={i:>3} | {frame}')
    i += 1
    
if frames_pesquisa[i_frame]:
    print(f'\n\nDados do teste: batch_name: {batch_name} | frame: {frames_pesquisa[i_frame]} | model: {model} | tipo_pdf: {tipo_pdf[i_test]}')
    
    
######### PATHS
#1. path formado para busca de pdfs recursiva
root_doc_analise = os.path.join(documentos_extracao_path, batch_name)

#2. path para documentos teste RASTER PDF (ATRIBIDO DA LISTA)
path_test_pdf = list_path_test[1]

#3. path formado para nome do arquivo json
json_file_path = os.path.join(json_path, nome_formado_json)

#Listando paths utilizados
#print(f'\nroot_doc_analise: {root_doc_analise}\npath_test_pdf: {path_test_pdf}\njson_file_path: {json_file_path}')       

seq =  0 | 1_frame_prefeitura_nf
seq =  1 | 1_frame_dados_nf
seq =  2 | 2_frame_cnpj_prestador
seq =  3 | 2_frame_inscricao_prestador
seq =  4 | 2_frame_dados_prestador
seq =  5 | 3_frame_cnpj_tomador
seq =  6 | 3_frame_inscricao_tomador
seq =  7 | 3_frame_dados_tomador
seq =  8 | 4_frame_descricao_totais
seq =  9 | 4_frame_valor_total
seq = 10 | 4_frame_cnae_itens_servico
seq = 11 | 5_frame_valores_impostos
seq = 12 | 5_frame_dados_complementares
seq = 13 | 5_frame_inf_criticas
seq = 14 | 5_frame_observacao


Dados do teste: batch_name: Batch_RASTER_PDF_0 | frame: 1_frame_prefeitura_nf | model: pedro_aldeia | tipo_pdf: RASTER_PDF


### Processamento do Pipeline 

In [50]:
root_doc_analise = "pipeline_extracao_documentos/2_documentos_para_extracao/Batch_11"

batch_name = 'Batch_11'

In [52]:
erros = {}

# 1. Leitura recursiva de diretorios e arquivos a partir de root
pdf_info = {}  # Dicionário para armazenar informações sobre PDFs

nf_data_servico = {}#VERIFICAR
analise_doc_nf = {} #VERIFICAR
file_data = [] #VERIFICAR

list_document_pages = []
#nro_nota = 0
# TEMP
# Nome do arquivo json
nome_formado_json = batch_name +".json"
#3. path formado para nome do arquivo json
json_file_path = os.path.join(json_path, nome_formado_json)
#root_doc_analise = os.path.join(documentos_extracao_path, batch_name)
#print(root_doc_analise)
i = 1
for root, dirs, files in os.walk(root_doc_analise):
    dir_name = os.path.basename(root)
    #print(dir_name)
    for file in files:
        
        if file.lower().endswith('.pdf'):
            doc2convert = file
            document_path_1 = os.path.join(root, file)
            pdf_document = fitz.open(document_path_1)
            #page_number = 0  # Defina o número da página que deseja analisar
            #page = pdf_document[page_number]
            
            documento_pdf = True
            pesquisavel, metadados, paginas = is_pdf_searchable_analise(document_path_1)
            print(f'\nTeste nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
                
            if pesquisavel:
                image_2work, name_image_2work = convertResizeAnalise_1page(file, document_path_1, image_resized_path)
                
                secao = "1 - CABECALHO"
                try:
                    nro_nota = 0
                    data_cabecalho = {}
                    data_cabecalho['secao'] = secao
                    valor_dict = {}
                    dados_prefeitura = {}
                    f_frame_name = "1_frame_prefeitura_nf"
                    # 1. funçao basica de modelo 
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text_splited = texto.split('\n')
                    
                    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
                    if valor_dict:
                        dados_prefeitura.update(valor_dict)
                    valor = {}   
                    f_frame_name = "1_frame_dados_nf"
                    dadinho_dados_nf = {}
                    # 1. funçao basica de modelo 
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

                    string_pesquisa = "Número da Nota:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
                    dadinho_dados_nf['numero_nota_fiscal'] = texto
                    nro_nota = texto
                    
                    string_pesquisa = "Competência:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['competencia'] = texto
                    
                    string_pesquisa = "ata e Hora da Emissão:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['dt_hr_emissao'] = texto
                    
                    string_pesquisa = "Código Verificação:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['codigo_verificacao'] = texto
                    
                    data_cabecalho.update(dados_prefeitura)
                    data_cabecalho.update(dadinho_dados_nf)
                except Exception as e:
                    erros_cabecalho = {}
                    err_msg = f"Erro de processo cabecalho: {e}"
                    erros['documento'] = file
                    erros_cabecalho['secao'] = secao
                    erros_cabecalho['erro'] = err_msg
                    erros.update(erros_cabecalho)                
               
                
                
                
                secao = "2. PRESTADOR DE SERVIÇO"
                try:
                    f_frame_name = "2_frame_cnpj_prestador"
                    data_prestador = {}
                    prestador_inscricao = {}
                    data_prestador['secao'] = secao
                    prestador_cnpj_value = {}
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    prestador_cnpj_value = extract_fields_prestador_cnpj(texto)
                    if prestador_cnpj_value:
                        data_prestador.update(prestador_cnpj_value)
                except Exception as e:
                    erros_cnpj_prestador = {}
                    err_msg = (f"Erro prestador cnpj: {e}")
                    erros_cnpj_prestador['secao'] = secao
                    erros_cnpj_prestador['erro'] = err_msg
                    erros.update(erros_cnpj_prestador)       
                    
                try:
                    f_frame_name = "2_frame_inscricao_prestador" 
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
                    string_pesquisa = "Inscrição Municipal:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_inscricao['prestador_inscricao'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_inscricao['inscricao_estadual'] = texto
                    data_prestador.update(prestador_inscricao)
                except Exception as e:
                    erros_inscricao_prestador = {}
                    err_msg = (f"Erro de processo inscricao prestador: {e}")
                    erros_inscricao_prestador['secao'] = secao
                    erros_inscricao_prestador['erro'] = err_msg
                    erros.update(erros_inscricao_prestador)

                try:
                    f_frame_name = "2_frame_dados_prestador"
                    prestador_dados_value = {}
                    
                    keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['razao_social'] = texto

                    string_pesquisa = "Nome de Fantasia:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['nome_fantasia'] = texto
                    
                    string_pesquisa = "Endereço:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['endereco'] = texto
                    
                    string_pesquisa = "E-mail:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['email'] = texto
                    data_prestador.update(prestador_dados_value)
                    
                except Exception as e:
                    print(f"Erro prestador dados: {e}") 
                    
                
                secao = "3. TOMADOR DE SERVIÇO"
                try:
                    data_tomador = {}
                    tomador_cnpj_value = {}
                    data_tomador['secao'] = secao
                    f_frame_name = "3_frame_cnpj_tomador"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto) 
                    tomador_cnpj_value = extract_fields_tomador_cnpj(texto)
                    if tomador_cnpj_value:
                        data_tomador.update(tomador_cnpj_value)                  
                except Exception as e:
                    print(f"Erro tomador cnpj: {e}")
                    
                f_frame_name = "3_frame_inscricao_tomador"    
                try:
                    data_tomador_inscricao = {}
                    keyword_list = ['RG:', 'Inscrição Estadual:']
                    string_pesquisa = "RG:"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_inscricao['rg'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_inscricao['inscricao_estadual'] = texto
                    data_tomador.update(data_tomador_inscricao)
                except Exception as e:
                    print(f"Erro tomador inscricao: {e}")
                    
                f_frame_name = "3_frame_dados_tomador"
                try: 
                    data_tomador_dados = {}   
                    keyword_list = ['Nome/Razão Social:', 'Endereço:', 'E-mail']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['razao_social'] = texto
                    
                    string_pesquisa = "Endereço:"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['endereco'] = texto
                    
                    string_pesquisa = "E-mail"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['email'] = texto
                    
                    data_tomador.update(data_tomador_dados)
                except Exception as e:
                    print(f"Erro tomador dados: {e}") 
                    
                secao = "4. DESCRIMINACAO DOS SERVIÇOS"
                try:
                    data_servico = {}
                    data_servico['secao'] = secao
                    f_frame_name = "4_frame_descricao_totais"
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text = texto.replace('\n', ' ')
                    label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                    if text.startswith(label):
                        text = text[len(label):].strip()
                    data_servico['discriminacao_servicos'] = text 

                except Exception as e:
                    print(f"Erro descricao servico: {e}")
                     
                secao = "5. VALOR TOTAL"
                try:
                    data_valor_total = {}
                    data_valor_total['secao'] = secao
                    f_frame_name = "4_frame_valor_total"   
                    text = executa_model_frame(model, secao, f_frame_name)  
                    valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
                    if valor_total_match:
                        valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                        data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
                except Exception as e:
                    print(f"Erro valor total: {e}")

                secao = "6. CNAE e Item da Lista de Serviços"
                try:
                    data_CNAE = {}
                    data_CNAE['secao'] = secao
                    f_frame_name = "4_frame_cnae_itens_servico"   
                    Texto_extraido = executa_model_frame(model, secao, f_frame_name)
                    text_splited = Texto_extraido.split('\n')
                    # Processando CNAE
                    cnae_line = [line for line in text_splited if 'CNAE' in line][0]
                    cnae_number = int(extract_number(cnae_line))
                    cnae_value = cnae_dict.get(cnae_number, "Valor não encontrado")
                    if cnae_value == 'Valor não encontrado':
                        cnae_value = processa_cnae_outros(cnae_line)
                        cnae_value = cnae_value.upper()
                        data_CNAE['cnae'] = cnae_value
                    else:
                        cnae_value = cnae_value.upper()
                        cnae_value = str(cnae_number) + " - " + cnae_value
                        data_CNAE['cnae'] = cnae_value
                        data_CNAE['item_lista_servicos'] = item_servico_value
                except Exception as e:
                    print(f"Erro busca cnae: {e}")    
          
                try:
                    item_servico_line = [line for line in text_splited if 'Item da Lista de Serviços' in line][0]
                    item_servico_number = float(extract_number(item_servico_line))
                    item_servico_value = item_servico_dict.get(item_servico_number, "Valor não encontrado")
                    item_servico_value = item_servico_value.upper()
                    item_servico_value = str(item_servico_number) + " - " + item_servico_value
                    data_CNAE['item_lista_servicos'] = item_servico_value
                except Exception as e:
                    print(f"Erro busca Itens de servico: {e}")  

                secao = "8. DADOS COMPLEMENTARES"
                try:
                    data_valores = {}
                    data_valores['secao'] = secao
                    f_frame_name = "5_frame_valores_impostos"   
                    
                    result = extract_fields_box(model, f_frame_name, secao)
                    if result:
                        data_valores.update(result)
            
                    # secao: 8 - DADOS COMPLEMENTARES"
                    data_dados_complementares = {}
                    f_frame_name  = "5_frame_dados_complementares"
                    section = "8. DADOS COMPLEMENTARES"

                    data_dados_complementares = extract_dados_comple_obs(model, f_frame_name, section)                                           
                                            
                                            
                    # secao: 9 - OUTRAS INFORMAÇOES / CRITICAS
                    data_outras_informacoes = {}
                    father_value = "5_frame_inf_criticas"
                    section = "9. OUTRAS INFORMAÇOES / CRITICAS"
                    
                    result = extract_fb_outras_inf(model, father_value, section)
                    if result:
                        data_outras_informacoes.update(result)                        
                                        
                    # secao: 10. OBSERVACOES
                    data_observacao = {}
                    f_father = "5_frame_observacao"
                    section = "10. OBSERVACOES"

                    data_observacao = extract_dados_comple_obs(model, f_father, section)
                except Exception as e:
                    print(f"Erro valores complementares: {e}")   
                
                nome_arquivo = file
                #pdf_info["diretorio"] = os.path.basename(root)
                try:
                    pdf_info[nro_nota] = {
                    "dados_NF_PDF": {
                        "data_cabecalho": data_cabecalho,
                        "data_prestador": data_prestador,
                        "data_tomador": data_tomador,
                        "data_servico": data_servico,
                        "data_valor_total": data_valor_total,
                        "data_CNAE": data_CNAE,
                        "data_valores": data_valores,
                        "data_dados_complementares": data_dados_complementares,
                        "data_outras_informacoes": data_outras_informacoes,
                        "data_observacao": data_observacao,
                    },
                    "diretorio": dir_name, #os.path.basename(root)
                    "nome_arquivo": nome_arquivo,
                    "Batch": batch_name,
                    "modelo": model,
                }
                except Exception as e:
                    print(f"Erro ao gerar o json: {e}")
                pdf_document.close()
                #print(pdf_info)
                #if paginas == 1:
                if paginas > 1000:
                    if i == 1000: #Define quantidade de tratamento de documentos raster PDF
                        break
            i +=1 
                
# Salvando as informações em um arquivo JSON (novo formato nome arquivo V2)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)

print(f"As informações foram salvas em {json_file_path}")

#print(pdf_info)  


Teste nro: 1 | doc: 1_5152158024281883471.pdf | pdf?: True | pesquisavel?: True | paginas: 1

67    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_dados_complementares nan                  148.0   2273.0  1925.0  2377.0 
76    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_observacao   uma observação       148.0   2515.0  1922.0  2676.0 

Teste nro: 2 | doc: 1_5152158024281883472.pdf | pdf?: True | pesquisavel?: True | paginas: 1

67    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_dados_complementares nan                  148.0   2273.0  1925.0  2377.0 
76    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_observacao   uma observação       148.0   2515.0  1922.0  2676.0 
As informações foram salvas em pipeline_extracao_documentos/5_documentos_processados/jsons/Batch_11.json


## Desenha Box

In [ ]:
name_image_2work = "pipeline_extracao_documentos/6_geral_administacao/images/processadas/Doria Marinho 0297 Raquel.pdf.jpg"

In [14]:
# To draw everything
trattempl. draw_box_model(model, boundaries_info, sections_info, frames_info, field_boxes_info)


[16684:16712:0830/140554.653428:ERROR:bus.cc(406)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[16684:16716:0830/140554.695931:ERROR:bus.cc(406)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[16684:16716:0830/140554.696123:ERROR:bus.cc(406)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[16684:16712:0830/140554.701749:ERROR:bus.cc(406)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[16684:16712:0830/140554.701814:ERROR:bus.cc(406)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[16684:16712:0830/140554.701826:ERROR:bus.cc(406)] Failed to connect to the bus: Could not parse server address: Unknown address t

In [ ]:
# To draw only boundaries and sections:
draw_box_model(modelo, boundaries_info, sections_info, draw_frames=False, draw_field_boxes=False)


In [ ]:
# To draw only field boxes:
draw_box_model(modelo, field_boxes_info=field_boxes_info, draw_boundaries=False, draw_sections=False, draw_frames=False)


# Outros

In [ ]:
# Sao iguais APAGAR ESSA
def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

def executa_model()
i_frame = 0
frames_pesquisa = []
filtered_frames_nf_v4_df = frames_nf_v4_df[frames_nf_v4_df['model'] == model]
for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_type = row_frame['type']
    frame_section = row_frame['section_json']
    frame_reference = row_frame['reference']
    frame_father = row_frame['father']
    #if frame_section == "1 - CABECALHO" and frame_type == "frame":
    if frame_section == "1 - CABECALHO":
        frame_father = row_frame['father']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "1_frame_prefeitura_nf" and frame_type == "sframe_field":
        #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
        elif frame_father == "1_frame_dados_nf" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "2. PRESTADOR DE SERVIÇO":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "2_frame_cnpj_prestador" and frame_type == "sframe_field":
        #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
        elif frame_father == "2_frame_inscricao_prestador" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
        elif frame_father == "2_frame_dados_prestador" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
    elif frame_section == "3. TOMADOR DE SERVIÇO":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            data_cabecalho = {}
            data_cabecalho['secao'] = frame_section
            if frame_father == "3_frame_cnpj_tomador" and frame_type == "sframe_field":
                #if frame_label == "1_frame_prefeitura_nf":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}
            
            elif frame_father == "3_frame_inscricao_tomador" and frame_type == "sframe_field":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {} 
            elif frame_father == "3_frame_dados_tomador" and frame_type == "sframe_field":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}
                
    elif frame_section == "4. DESCRIMINACAO DOS SERVIÇOS":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            data_cabecalho = {}
            data_cabecalho['secao'] = frame_section
            if frame_father == "4_frame_descricao_totais" and frame_type == "field_box":
                #if frame_label == "1_frame_prefeitura_nf":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}  
                
    elif frame_section == "5. VALOR TOTAL":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "4_frame_valor_total" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}   
            
    elif frame_section == "6. CNAE e Item da Lista de Serviços":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "4_frame_cnae_itens_servico" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
    
    elif frame_section == "8. DADOS COMPLEMENTARES":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_valores_impostos" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
        if frame_father == "5_frame_dados_complementares" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "9. OUTRAS INFORMAÇOES / CRITICAS":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_inf_criticas" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "10. OBSERVACOES":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_observacao" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}                             
        

In [ ]:
def executa_model()
i_frame = 0
frames_pesquisa = []
filtered_frames_nf_v4_df = frames_nf_v4_df[frames_nf_v4_df['model'] == model]
for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_type = row_frame['type']
    frame_section = row_frame['section_json']
    frame_reference = row_frame['reference']
    frame_father = row_frame['father']
    #if frame_section == "1 - CABECALHO" and frame_type == "frame":
    if frame_section == "1 - CABECALHO":
        frame_father = row_frame['father']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "1_frame_prefeitura_nf" and frame_type == "sframe_field":
        #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
        elif frame_father == "1_frame_dados_nf" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "2. PRESTADOR DE SERVIÇO":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "2_frame_cnpj_prestador" and frame_type == "sframe_field":
        #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
        elif frame_father == "2_frame_inscricao_prestador" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
        elif frame_father == "2_frame_dados_prestador" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
    elif frame_section == "3. TOMADOR DE SERVIÇO":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            data_cabecalho = {}
            data_cabecalho['secao'] = frame_section
            if frame_father == "3_frame_cnpj_tomador" and frame_type == "sframe_field":
                #if frame_label == "1_frame_prefeitura_nf":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}
            
            elif frame_father == "3_frame_inscricao_tomador" and frame_type == "sframe_field":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {} 
            elif frame_father == "3_frame_dados_tomador" and frame_type == "sframe_field":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}
                
    elif frame_section == "4. DESCRIMINACAO DOS SERVIÇOS":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            data_cabecalho = {}
            data_cabecalho['secao'] = frame_section
            if frame_father == "4_frame_descricao_totais" and frame_type == "field_box":
                #if frame_label == "1_frame_prefeitura_nf":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}  
                
    elif frame_section == "5. VALOR TOTAL":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "4_frame_valor_total" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}   
            
    elif frame_section == "6. CNAE e Item da Lista de Serviços":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "4_frame_cnae_itens_servico" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
    
    elif frame_section == "8. DADOS COMPLEMENTARES":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_valores_impostos" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
        if frame_father == "5_frame_dados_complementares" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "9. OUTRAS INFORMAÇOES / CRITICAS":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_inf_criticas" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "10. OBSERVACOES":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_observacao" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}                             
        
        